# Poly Expo go BRRRRR

In this challenge, `n` is an RSA key and `N` is its polynomial in `Z/2Z`, with bits of `n` as coefficients.
`C = M^e % N`, where `M` is the polynomial of `msg` and `C` the polynomial of `c`.

In [1]:
n = 119688104315557021890936576297322528053073582644938225605833562855944546643311189725353580415278613605803528999976536949698525581164157480218289586687945087549976509446759942778609918817975151644563678567137671925049937536315926169828583738712154203276012477308556625213229949900385215601055758028238785190211
e = 65537
c = 59180475538014020769986137847579404920412136380976726613826924727288568855214946199702335444771145318394201684142700441287649150098774979773106915707593238156979003572684188994666984941867671144226245449471326607224512384706414018555885923177955268177207582929765645093722741174664225408159262482249199006862

P.<x> = GF(2)[]
N = sum([int(b) * x^i for i, b in enumerate(bin(n)[2:])])
Q.<x> = P.quotient(N)

In [2]:
C = sum(int(b) * P.gen()^i for i, b in enumerate(f'{c:01023b}'))

In [3]:
def encrypt(msg):
    if type(msg) == bytes:
        msg = int.from_bytes(msg, 'big')

    assert msg < n
    msg = sum([int(b) * x^i for i, b in enumerate(f'{msg:01023b}')])
    cip = Q(msg)^e
    cip = int(''.join([str(i) for i in list(cip)]), 2)
    return cip

An interesting thing to know is that it's much easier to factor `N` than it is to factor `n`. As such, we will find the 65537th root of `M` over `K[x]/(f^i)` for each factor `f` of `N`, then combine them with the chinese remainder theorem.

NB: This theorem, while usually presented as a result over the integers, holds over all euclidean rings (such as `K[x]`).

NB 2: It even holds over principal ideal domains, but non-constructively so we can't compute solutions in that case.

In [4]:
facs = list(N.factor())

In [5]:
# When f is irreducible over K[x], K[x]/(f) is a field
Qi = [P.quotient(fac^exp, names='x') if exp > 1 else GF(2^fac.degree(), modulus=fac, name='x') for fac, exp in facs]
Qi

[Univariate Quotient Polynomial Ring in x over Finite Field of size 2 with modulus x^5 + x^4 + x + 1,
 Finite Field in x of size 2^3,
 Finite Field in x of size 2^7,
 Finite Field in x of size 2^24,
 Finite Field in x of size 2^85,
 Finite Field in x of size 2^257,
 Finite Field in x of size 2^642]

In [6]:
# Sage has builtin nth-root for fields, but not for quotient polynomial rings
roots = [F(C).nth_root(e) for F in Qi[1:]]

In [7]:
Qi[0].order()

32

Since the order of the ring is small, we can bruteforce all polynomials to find the 65537th root of `C`.

In [8]:
C0 = Qi[0](C)
for root0 in Qi[0]:
    if root0^e == C0:
        break

root0

x^3 + x^2 + x

In [9]:
# Lift the polynomials from K[x]/(f) to K[x]
residues = [sum(int(c) * P.gen()^i for i, c in enumerate(root0))] + [P(root) for root in roots]
residues[:4]

[x^3 + x^2 + x,
 x^2 + x,
 x^5 + x + 1,
 x^21 + x^20 + x^18 + x^17 + x^15 + x^13 + x^12 + x^10 + x^8 + x^7 + x^6 + x^4 + x^3 + x + 1]

In [10]:
F = crt(residues, [F.modulus() for F in Qi])

In [11]:
sum(int(bit) << i for i, bit in enumerate(F.list()[::-1])).to_bytes(90, 'big')

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00flag{bRrRrRrRr_Yun_C4nt0r_4nd_Z4ss3nh4us_34t_p0lyn0m14ls_f0r_br34kf4st_bRrRrRrRr}'